## Investigating the Zillow Housing Price Dataset

N. Ranjan & R. Mattson | CSCI 6430* | Mar 11, 2021




TODO: 
- check file names

In [177]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error as mape
#from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline


###  Retrieve Data

In [178]:
#TODO - get from google docs??

DF1B = pd.read_csv(r'Datasets/City_MedianRentalPrice_1Bedroom.csv')
DF2B = pd.read_csv(r'Datasets/City_MedianRentalPrice_2Bedroom.csv')
DF3B = pd.read_csv(r'Datasets/City_MedianRentalPrice_3Bedroom.csv')
DF4B = pd.read_csv(r'Datasets/City_MedianRentalPrice_4Bedroom.csv')

In [179]:
DF1B.columns

Index(['Unnamed: 0', 'RegionName', 'State', 'Metro', 'CountyName', 'SizeRank',
       '2010-02', '2010-03', '2010-04', '2010-05',
       ...
       '2019-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08',
       '2019-09', '2019-10', '2019-11', '2019-12'],
      dtype='object', length=125)

In [180]:
# remove columns with useless information
DF1B.drop(columns=['SizeRank','Unnamed: 0'],inplace=True)
DF2B.drop(columns=['SizeRank','Unnamed: 0'],inplace=True)
DF3B.drop(columns=['SizeRank','Unnamed: 0'],inplace=True)
DF4B.drop(columns=['SizeRank','Unnamed: 0'],inplace=True)

In [181]:
# record the number of bedrooms within the frame
DF1B.insert(4, 'Bedrooms', 1)
DF2B.insert(4, 'Bedrooms', 2)
DF3B.insert(4, 'Bedrooms', 3)
DF4B.insert(4, 'Bedrooms', 4)

In [182]:
frames = [DF1B,DF2B,DF3B,DF4B]
result = pd.concat(frames)
result = result.sample(frac=1) #Shuffle!
result = result.reset_index(drop=True) #Reset Index

In [183]:
result

,RegionName,State,Metro,CountyName,Bedrooms,2010-02,2010-03,2010-04,2010-05,2010-06,...,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
0,Oshkosh,WI,Oshkosh-Neenah,Winnebago County,2,NaN,NaN,NaN,NaN,NaN,...,725.0,750.0,720.0,700.0,745.0,747.5,725.0,740.0,740.0,725.0
1,Norristown,PA,Philadelphia-Camden-Wilmington,Montgomery County,2,NaN,NaN,NaN,NaN,NaN,...,1397.0,1399.0,1350.0,1360.0,1375.0,1391.0,1382.5,1450.0,1497.5,1475.0
2,Annapolis,MD,Baltimore-Columbia-Towson,Anne Arundel County,1,NaN,NaN,NaN,NaN,NaN,...,1655.0,1650.0,1680.0,1675.0,1675.0,1602.5,1584.0,1562.0,1575.0,1580.0
3,Hutchinson Island South,FL,Port St. Lucie,Saint Lucie County,2,NaN,NaN,NaN,NaN,NaN,...,2950.0,2800.0,2800.0,2800.0,2800.0,2900.0,2800.0,3000.0,3050.0,3200.0
4,Newton,MA,Boston-Cambridge-Newton,Middlesex County,2,NaN,NaN,NaN,NaN,NaN,...,2375.0,2375.0,2400.0,2400.0,2400.0,2500.0,2400.0,2400.0,2400.0,2300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3384,Keller,TX,Dallas-Fort Worth-Arlington,Tarrant County,3,NaN,NaN,NaN,NaN,NaN,...,1795.0,1892.5,2000.0,1800.0,1740.0,1749.5,1825.0,1697.5,1689.5,1697.0
3385,Columbia,SC,Columbia,Richland County,4,NaN,NaN,NaN,NaN,NaN,...,1400.0,1425.0,1412.5,1364.5,1449.5,1400.0,1300.0,1349.5,1400.0,1400.0
3386,Middletown,OH,Cincinnati,Butler County,2,NaN,NaN,NaN,NaN,NaN,...,755.0,775.0,785.0,795.0,750.0,737.5,790.0,800.0,745.0,750.0
3387,Gainesville,FL,Gainesville,Alachua County,3,NaN,NaN,NaN,NaN,NaN,...,1495.0,1500.0,1479.0,1490.0,1475.0,1490.0,1409.0,1400.0,1425.0,1450.0


In [184]:
# result.to_csv(r'Datasets\City_MedianRentalPrice_AllHomes.csv')
result.to_csv(r'Datasets/City_MedianRentalPrice_AllHomes_ALTERED.csv',index=False)



In [185]:
# dels

### Preproccess Data

In [186]:
# DF = pd.read_csv(r'Datasets\City_MedianRentalPrice_AllHomes.csv')
DF = pd.read_csv(r'Datasets/City_MedianRentalPrice_AllHomes_ALTERED.csv')

In [187]:
print(DF.isnull().sum().to_numpy())

[   0    0    0    0    0 3365 3352 3348 3346 3346 3337 3328 3306 3287
 3280 3271 3136 3016 2990 2972 2932 2887 2802 2760 2723 2675 2646 2634
 2604 2587 2575 2561 2548 2538 2526 2519 2514 2500 2493 2483 2479 2472
 2462 2459 2456 2452 2452 2448 2445 2431 2419 2419 2402 2388 2385 2384
 2382 2365 2359 2349 2346 2337 2333 2321 2312 2305 2297 2288 2284 2279
 2273 2267 2261 2258 2254 2245 2240 2235 2231 2225 2213 2185 2173 2154
 2121 2084 2020 1907 1400 1206 1109 1047  963  913  867  829  800  771
  741  695  621  571  510  465  412  361  306  249  209  144  107   60
    0    0    0    0    0    0    0    0    0    0    0    0]


In [188]:
cut_off = #why

SyntaxError: invalid syntax (<ipython-input-188-1a4325d0907d>, line 1)

In [189]:
# DF_Filt = DF.iloc[:,cut_off:-1]
DF_Filt = DF.iloc[:,-16:-1]
DF_Filt # Contains all numerical cols we want
for x in DF_Filt.columns.to_numpy():
   #sc = StandardScaler()
   DF_Filt[x] = DF_Filt[x].values.reshape(-1,1)
  
DFLblColumns = DF.iloc[:,0:5]
print(DFLblColumns.columns)
for x in DFLblColumns.columns.to_numpy():
   le = LabelEncoder() 
   DF_Filt[x] = le.fit_transform(DFLblColumns[x].values)

Index(['RegionName', 'State', 'Metro', 'CountyName', 'Bedrooms'], dtype='object')


In [190]:
Y = DF.iloc[:,-1]
Y

0        725.0
1       1475.0
2       1580.0
3       3200.0
4       2300.0
         ...  
3384    1697.0
3385    1400.0
3386     750.0
3387    1450.0
3388    1300.0
Name: 2019-12, Length: 3389, dtype: float64

In [191]:
X = DF_Filt
X

,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,RegionName,State,Metro,CountyName,Bedrooms
0,795.0,725.0,747.5,725.0,712.5,700.0,725.0,750.0,720.0,700.0,745.0,747.5,725.0,740.0,740.0,902,48,208,436,1
1,1445.0,1422.0,1445.0,1355.0,1350.0,1366.0,1397.0,1399.0,1350.0,1360.0,1375.0,1391.0,1382.5,1450.0,1497.5,843,38,214,265,1
2,1680.0,1680.0,1690.0,1615.0,1600.0,1630.0,1655.0,1650.0,1680.0,1675.0,1675.0,1602.5,1584.0,1562.0,1575.0,27,20,19,10,0
3,2800.0,2800.0,3000.0,3200.0,3200.0,3000.0,2950.0,2800.0,2800.0,2800.0,2800.0,2900.0,2800.0,3000.0,3050.0,556,9,219,351,1
4,2300.0,2250.0,2300.0,2300.0,2222.5,2250.0,2375.0,2375.0,2400.0,2400.0,2400.0,2500.0,2400.0,2400.0,2400.0,835,19,29,256,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3384,1697.5,1662.5,1700.0,1895.0,1834.0,1745.0,1795.0,1892.5,2000.0,1800.0,1740.0,1749.5,1825.0,1697.5,1689.5,592,43,66,400,2
3385,1295.0,1350.0,1400.0,1375.0,1399.0,1375.0,1400.0,1425.0,1412.5,1364.5,1449.5,1400.0,1300.0,1349.5,1400.0,235,40,61,332,3
3386,735.0,725.0,725.0,725.0,735.0,735.0,755.0,775.0,785.0,795.0,750.0,737.5,790.0,800.0,745.0,755,35,54,50,1
3387,1350.0,1400.0,1400.0,1400.0,1475.0,1500.0,1495.0,1500.0,1479.0,1490.0,1475.0,1490.0,1409.0,1400.0,1425.0,434,9,101,2,2


In [192]:
print("Number of nulls per column", X.isnull().sum())
for col in X:
    X.fillna(X[col].median(),inplace=True)
    #X = X.fillna(X[col].median())
    
print("After Replacing Null:")   
print("Max amount on nulls per column", X.isnull().sum().max())

Number of nulls per column 2018-09       209
2018-10       144
2018-11       107
2018-12        60
2019-01         0
2019-02         0
2019-03         0
2019-04         0
2019-05         0
2019-06         0
2019-07         0
2019-08         0
2019-09         0
2019-10         0
2019-11         0
RegionName      0
State           0
Metro           0
CountyName      0
Bedrooms        0
dtype: int64
After Replacing Null:
Max amount on nulls per column 0


### Run Models

### Linear Regression

In [194]:
model = make_pipeline(StandardScaler(),LinearRegression())

# Y_pred = model.predict(X_test)
# Y_pred = scY.inverse_transform(Y_pred)
# Y_test = scY.inverse_transform(Y_test)


scores = cross_validate(model, X, Y, cv=10,scoring=('r2', 'neg_root_mean_squared_error','neg_mean_absolute_percentage_error'))
meanR2 = np.absolute(scores['test_r2']).mean()
medianR2 = np.median(np.absolute(scores['test_r2']))
meanRMSE = np.absolute(scores['test_neg_root_mean_squared_error']).mean()
medianRMSE = np.median(np.absolute(scores['test_neg_root_mean_squared_error']))
meanMAPE = np.absolute(scores['test_neg_mean_absolute_percentage_error']).mean()
medianMAPE = np.median(np.absolute(scores['test_neg_mean_absolute_percentage_error']))

In [195]:
print('Average R2 score ', meanR2)
print('Median R2 score ', medianR2)
print('Average Root Mean Square Error ', meanRMSE)
print('Median Root Mean Square Error ', medianRMSE)
print('Average Mean Absolute Percentage Error ', meanMAPE)
print('Median Mean Absolute Percentage Error ', medianMAPE)

Average R2 score  0.9932104174998116
Median R2 score  0.993506237546615
Average Root Mean Square Error  141.10762758543027
Median Root Mean Square Error  106.22689409783752
Average Mean Absolute Percentage Error  0.03048792778833856
Median Mean Absolute Percentage Error  0.03101421765014442


In [196]:
# Peek at learned parameters
model_temp = LinearRegression() 
hist = model_temp.fit(X,Y)
model_temp.coef_

array([-2.96921679e-02,  5.63445383e-02,  4.03086728e-02,  4.55865670e-02,
       -2.01418656e-01,  2.78328756e-01,  1.13017379e-01, -3.76380715e-02,
        5.53042426e-02, -4.25627877e-01,  7.76334543e-02,  1.97798434e-01,
        2.38308306e-01, -1.22746688e-01,  7.52773066e-01, -9.77964611e-04,
        4.82018012e-01, -4.85134916e-02, -5.33290956e-03, -8.35181036e+00])

Some parameter are producing very small parameter values, indicating that nothing useful is being learned.... I protest this 

In [197]:
X.drop(columns=['Metro', 'CountyName'], inplace=True)
X

,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,RegionName,State,Bedrooms
0,795.0,725.0,747.5,725.0,712.5,700.0,725.0,750.0,720.0,700.0,745.0,747.5,725.0,740.0,740.0,902,48,1
1,1445.0,1422.0,1445.0,1355.0,1350.0,1366.0,1397.0,1399.0,1350.0,1360.0,1375.0,1391.0,1382.5,1450.0,1497.5,843,38,1
2,1680.0,1680.0,1690.0,1615.0,1600.0,1630.0,1655.0,1650.0,1680.0,1675.0,1675.0,1602.5,1584.0,1562.0,1575.0,27,20,0
3,2800.0,2800.0,3000.0,3200.0,3200.0,3000.0,2950.0,2800.0,2800.0,2800.0,2800.0,2900.0,2800.0,3000.0,3050.0,556,9,1
4,2300.0,2250.0,2300.0,2300.0,2222.5,2250.0,2375.0,2375.0,2400.0,2400.0,2400.0,2500.0,2400.0,2400.0,2400.0,835,19,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3384,1697.5,1662.5,1700.0,1895.0,1834.0,1745.0,1795.0,1892.5,2000.0,1800.0,1740.0,1749.5,1825.0,1697.5,1689.5,592,43,2
3385,1295.0,1350.0,1400.0,1375.0,1399.0,1375.0,1400.0,1425.0,1412.5,1364.5,1449.5,1400.0,1300.0,1349.5,1400.0,235,40,3
3386,735.0,725.0,725.0,725.0,735.0,735.0,755.0,775.0,785.0,795.0,750.0,737.5,790.0,800.0,745.0,755,35,1
3387,1350.0,1400.0,1400.0,1400.0,1475.0,1500.0,1495.0,1500.0,1479.0,1490.0,1475.0,1490.0,1409.0,1400.0,1425.0,434,9,2


In [198]:
scores = cross_validate(model, X, Y, cv=10,scoring=('r2', 'neg_root_mean_squared_error','neg_mean_absolute_percentage_error'))
meanR2 = np.absolute(scores['test_r2']).mean()
medianR2 = np.median(np.absolute(scores['test_r2']))
meanRMSE = np.absolute(scores['test_neg_root_mean_squared_error']).mean()
medianRMSE = np.median(np.absolute(scores['test_neg_root_mean_squared_error']))
meanMAPE = np.absolute(scores['test_neg_mean_absolute_percentage_error']).mean()
medianMAPE = np.median(np.absolute(scores['test_neg_mean_absolute_percentage_error']))
print('Average R2 score ', meanR2)
print('Median R2 score ', medianR2)
print('Average Root Mean Square Error ', meanRMSE)
print('Median Root Mean Square Error ', medianRMSE)
print('Average Mean Absolute Percentage Error ', meanMAPE)
print('Median Mean Absolute Percentage Error ', medianMAPE)

Average R2 score  0.9932108270638367
Median R2 score  0.9935127998426236
Average Root Mean Square Error  141.0837826577377
Median Root Mean Square Error  106.21546321944973
Average Mean Absolute Percentage Error  0.03046901886997212
Median Mean Absolute Percentage Error  0.031116826601925954


wait yeah dropping useless columns did nothing duh. 

Drop all categorical data? 

In [200]:
X.drop(columns=['RegionName', 'State'], inplace=True)
X

,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,Bedrooms
0,795.0,725.0,747.5,725.0,712.5,700.0,725.0,750.0,720.0,700.0,745.0,747.5,725.0,740.0,740.0,1
1,1445.0,1422.0,1445.0,1355.0,1350.0,1366.0,1397.0,1399.0,1350.0,1360.0,1375.0,1391.0,1382.5,1450.0,1497.5,1
2,1680.0,1680.0,1690.0,1615.0,1600.0,1630.0,1655.0,1650.0,1680.0,1675.0,1675.0,1602.5,1584.0,1562.0,1575.0,0
3,2800.0,2800.0,3000.0,3200.0,3200.0,3000.0,2950.0,2800.0,2800.0,2800.0,2800.0,2900.0,2800.0,3000.0,3050.0,1
4,2300.0,2250.0,2300.0,2300.0,2222.5,2250.0,2375.0,2375.0,2400.0,2400.0,2400.0,2500.0,2400.0,2400.0,2400.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3384,1697.5,1662.5,1700.0,1895.0,1834.0,1745.0,1795.0,1892.5,2000.0,1800.0,1740.0,1749.5,1825.0,1697.5,1689.5,2
3385,1295.0,1350.0,1400.0,1375.0,1399.0,1375.0,1400.0,1425.0,1412.5,1364.5,1449.5,1400.0,1300.0,1349.5,1400.0,3
3386,735.0,725.0,725.0,725.0,735.0,735.0,755.0,775.0,785.0,795.0,750.0,737.5,790.0,800.0,745.0,1
3387,1350.0,1400.0,1400.0,1400.0,1475.0,1500.0,1495.0,1500.0,1479.0,1490.0,1475.0,1490.0,1409.0,1400.0,1425.0,2


In [175]:
scores = cross_validate(model, X, Y, cv=10,scoring=('r2', 'neg_root_mean_squared_error','neg_mean_absolute_percentage_error'))
meanR2 = np.absolute(scores['test_r2']).mean()
medianR2 = np.median(np.absolute(scores['test_r2']))
meanRMSE = np.absolute(scores['test_neg_root_mean_squared_error']).mean()
medianRMSE = np.median(np.absolute(scores['test_neg_root_mean_squared_error']))
meanMAPE = np.absolute(scores['test_neg_mean_absolute_percentage_error']).mean()
medianMAPE = np.median(np.absolute(scores['test_neg_mean_absolute_percentage_error']))
print('Average R2 score ', meanR2)
print('Median R2 score ', medianR2)
print('Average Root Mean Square Error ', meanRMSE)
print('Median Root Mean Square Error ', medianRMSE)
print('Average Mean Absolute Percentage Error ', meanMAPE)
print('Median Mean Absolute Percentage Error ', medianMAPE)

Average R2 score  0.9924965786233525
Median R2 score  0.9944905913310678
Average Root Mean Square Error  135.58923816569674
Median Root Mean Square Error  102.78226330676563
Average Mean Absolute Percentage Error  0.030530779075527027
Median Mean Absolute Percentage Error  0.03136475925310139


I got no improvement.... nikhil wym??? 

### Polynomial Interpolation

In [201]:
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

for _,degree in enumerate([1,2,3]):
    polymodel = make_pipeline(StandardScaler(),PolynomialFeatures(degree),Ridge(alpha=1e-3))
    scores = cross_validate(polymodel, X, Y, cv=10,scoring=('r2', 'neg_root_mean_squared_error','neg_mean_absolute_percentage_error'))
    print('-'*5)
    print('Polynomial Degree: '+str(degree))
    print('-'*5)
    meanR2 = np.absolute(scores['test_r2']).mean()
    medianR2 = np.median(np.absolute(scores['test_r2']))
    meanRMSE = np.absolute(scores['test_neg_root_mean_squared_error']).mean()
    medianRMSE = np.median(np.absolute(scores['test_neg_root_mean_squared_error']))
    meanMAPE = np.absolute(scores['test_neg_mean_absolute_percentage_error']).mean()
    medianMAPE = np.median(np.absolute(scores['test_neg_mean_absolute_percentage_error']))
    print('Average R2 score ',meanR2)
    print('Median R2 score ',medianR2)
    print('Average Root Mean Square Error ',meanRMSE)
    print('Median Root Mean Square Error ',medianRMSE)
    print('Average Mean Absolute Percentage Error ',meanMAPE)
    print('Median Mean Absolute Percentage Error ',medianMAPE)    
    print('-'*10)
    

-----
Polynomial Degree: 1
-----
Average R2 score  0.9932138559480557
Median R2 score  0.993603335676336
Average Root Mean Square Error  141.04045951327853
Median Root Mean Square Error  107.04802429994103
Average Mean Absolute Percentage Error  0.0304073984403133
Median Mean Absolute Percentage Error  0.03080516161575434
----------
-----
Polynomial Degree: 2
-----
Average R2 score  0.8721595797466781
Median R2 score  0.9649193790988893
Average Root Mean Square Error  813.4179797186036
Median Root Mean Square Error  416.40212608851755
Average Mean Absolute Percentage Error  0.025454089240173155
Median Mean Absolute Percentage Error  0.025078373354244086
----------
-----
Polynomial Degree: 3
-----
Average R2 score  356.6910159106502
Median R2 score  24.24648413165005
Average Root Mean Square Error  22914.25685027387
Median Root Mean Square Error  7625.263005916353
Average Mean Absolute Percentage Error  0.06312640150830037
Median Mean Absolute Percentage Error  0.04332773403985189
-----

In [ ]:
## WE should not be doing R^2 score, should we?